In [0]:
# Install dependencies
# Agent libs used to deploy to databricks
%pip install -U -qqqq mlflow backoff databricks-openai databricks-agents uv

In [0]:
# For reference, working versions for installation:
# %md
# ``````bash
# Collecting brickops==0.3.16
#   Downloading brickops-0.3.16-py3-none-any.whl.metadata (7.6 kB)
# Collecting pyyaml>=6.0.2 (from brickops==0.3.16)
#   Downloading PyYAML-6.0.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
# Collecting requests>=2.32.3 (from brickops==0.3.16)
#   Downloading requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
# Requirement already satisfied: charset-normalizer<4,>=2 in /databricks/python3/lib/python3.11/site-packages (from requests>=2.32.3->brickops==0.3.16) (2.0.4)
# Requirement already satisfied: idna<4,>=2.5 in /databricks/python3/lib/python3.11/site-packages (from requests>=2.32.3->brickops==0.3.16) (3.4)
# Requirement already satisfied: urllib3<3,>=1.21.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-87b3d706-cbfa-4b1c-bd8c-e065b8719bbf/lib/python3.11/site-packages (from requests>=2.32.3->brickops==0.3.16) (2.0.7)
# Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.11/site-packages (from requests>=2.32.3->brickops==0.3.16) (2023.7.22)
# Downloading brickops-0.3.16-py3-none-any.whl (28 kB)
# Downloading PyYAML-6.0.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (762 kB)
#    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/763.0 kB ? eta -:--:--
#    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.0/763.0 kB 49.3 MB/s eta 0:00:00
# Downloading requests-2.32.3-py3-none-any.whl (64 kB)
# Installing collected packages: requests, pyyaml, brickops
#   Attempting uninstall: requests
#     Found existing installation: requests 2.31.0
#     Not uninstalling requests at /databricks/python3/lib/python3.11/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-87b3d706-cbfa-4b1c-bd8c-e065b8719bbf
#     Can't uninstall 'requests'. No files were found to uninstall.
#   Attempting uninstall: pyyaml
#     Found existing installation: PyYAML 6.0
#     Not uninstalling pyyaml at /databricks/python3/lib/python3.11/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-87b3d706-cbfa-4b1c-bd8c-e065b8719bbf
#     Can't uninstall 'PyYAML'. No files were found to uninstall.
# Successfully installed brickops-0.3.16 pyyaml-6.0.2 requests-2.32.3
# Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
# ``````

In [0]:
# Brickbots used for dataops and naming
%pip install brickops=0.3.16

In [0]:
dbutils.library.restartPython()

In [0]:
# Enable live reloading of libs, not needed now
%load_ext autoreload
%autoreload 2

In [0]:
import mlflow # works
# Not working # from databricks import mlflow
import os

In [0]:
# Get databricks host
BOTOPS_DATABRICKS_URL = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().get()
BOTOPSGENIE_DATABRICKS_HOST = BOTOPS_DATABRICKS_URL[8:]  # chop off https://

In [0]:
# Setup credentials used to access genie space in bot
#
# Set secret_scope_name and secret_key_name to access your Genie space
# We need the env vars to pass the values to the bot for deployment.

secret_scope_name = "botops"
os.environ["BOTOPSGENIE_SERVICE_PRINCIPAL_ID"] = dbutils.secrets.get(
    scope=secret_scope_name, key="botopsgenie_service_principal_id"
)
assert os.environ["BOTOPSGENIE_SERVICE_PRINCIPAL_ID"] is not None, (
    "The BOTOPSGENIE_SERVICE_PRINCIPAL_ID was not properly set"
)

os.environ["BOTOPSGENIE_SERVICE_PRINCIPAL_SECRET"] = dbutils.secrets.get(
    scope=secret_scope_name, key="botopsgenie_service_secret"
)
assert os.environ["BOTOPSGENIE_SERVICE_PRINCIPAL_SECRET"] is not None, (
    "The BOTOPSGENIE_SERVICE_PRINCIPAL_SECRET was not properly set"
)

os.environ["BOTOPSGENIE_SPACE_ID"] = dbutils.secrets.get(
    scope=secret_scope_name, key="botopsgenie_geniespace_id"
)
assert os.environ["BOTOPSGENIE_SPACE_ID"] is not None, (
    "The BOTOPSGENIE_SPACE_ID was not properly set"
)

os.environ["BOTOPSGENIE_DATABRICKS_HOST"] = BOTOPSGENIE_DATABRICKS_HOST
assert os.environ["BOTOPSGENIE_DATABRICKS_HOST"] is not None, (
    "The BOTOPSGENIE_DATABRICKS_HOST was not properly set"
)


In [0]:
# Used by agents.deploy()
BOTOPS_ENV_VARS = {
    "BOTOPSGENIE_SERVICE_PRINCIPAL_ID": os.environ["BOTOPSGENIE_SERVICE_PRINCIPAL_ID"],
    "BOTOPSGENIE_SERVICE_PRINCIPAL_SECRET": os.environ["BOTOPSGENIE_SERVICE_PRINCIPAL_SECRET"],
    "BOTOPSGENIE_SPACE_ID": os.environ["BOTOPSGENIE_SPACE_ID"],
    "BOTOPSGENIE_DATABRICKS_HOST": os.environ["BOTOPSGENIE_DATABRICKS_HOST"],
}

In [0]:
def folder():
    import os
    return os.path.basename(os.getcwd())